<a href="https://colab.research.google.com/github/ARAN-ai-python/RealEstateRentPrediction_AI/blob/main/jupyter%20notebook/%E4%B8%8D%E5%8B%95%E7%94%A3%E8%B3%83%E6%96%99%E4%BA%88%E6%B8%ACAI(SUUMO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
from time import sleep
from pprint import pprint
import json

import numpy as np
import pandas as pd
pd.set_option('display.max_rows',None)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import xgboost as xgb

In [ ]:
#endは偶数
start = 1
end = 20
place = '相模原'

urls = {
    '相模原':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '渋谷':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13113&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}",
    '新宿':"https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=13&sc=13104&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn={}"
}
#urls[place]
for i in range(start,end+1,2):
    print(i)
    
for i in range(start+1,end+1,2):
    print(i)

1
3
5
7
9
11
13
15
17
19
2
4
6
8
10
12
14
16
18
20


In [ ]:
#相模原orの物件の賃料データスクレイピングの関数
#Real_Estate_Data_Scraping
def REDS(start,end,place):
    
    #初期値
    d_list = []
    url = urls[place]
    
    for i in range(start,end+1,2):
    
        #ページを遷移させる
        target_url = url.format(i)

        #Requestsを用いてtarget_urlにアクセスする
        r = requests.get(target_url)

        #サーバー負荷軽減の為、ループ毎に1秒間隔を空ける
        sleep(1)

        #取得したHTMLをBeautifulSoupで解析する
        soup = BeautifulSoup(r.text)

        contents = soup.find_all('div',class_='cassettebox js-normalLink js-cassetLink')
        for content in contents:

            rows = content.find_all('table',class_='listtable')
            #title = content.find('a',class_='js-cassetLinkHref').text.split()[1]
            address = rows[0].find_all('div',class_='infodatabox-box-txt')[0].text
            station = rows[0].find_all('div',class_='infodatabox-box-txt')[1].text
            access = rows[0].find_all('div',class_='infodatabox-box-txt')[2].text

            #下の行のデータを取得
            r_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[0].text[:-2]
            mc_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[1].text[:-1]
            k_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[2].text.split('/')[0]
            s_fees = rows[1].find_all('dd',class_='infodatabox-details-txt')[2].text.split('/')[1][:-2]
            area = rows[1].find_all('dd',class_='infodatabox-details-txt')[3].text[:-2]
            layout = rows[1].find_all('dd',class_='infodatabox-details-txt')[4].text
            age = rows[1].find_all('div',class_='infodatabox-box-txt')[2].text

            #取得した各種データを辞書dに格納する
            d = {
            '住所':address,
            '路線':station,
            '交通':access,
            '賃料':r_fees,
            '管理共益費':mc_fees,
            '礼金':k_fees,
            '敷金':s_fees,
            '専有面積':area,
            '間取り':layout,
            '築年数':age
            }

            #辞書dのデータをリストd_listに格納する
            d_list.append(d)
            
            #重複したデータを削除する
            d_list = list(map(json.loads,set(map(json.dumps,d_list))))

        #進捗を報告させる
        print("d_list's progress:",i,"page　　",len(d_list))
        print(target_url)
        
        #データ数が100以上でない場合、スクレイピングを延長する
        #if(((i==end-1) or (i==end)) and (len(d_list) < 100)):
            #print('extended!')
            #end+=2
    
    print('Scraping Completed!')
    return d_list


In [ ]:
d_list = REDS(start,end,place)

d_list's progress: 1 page　　 24
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=1
d_list's progress: 3 page　　 46
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=3
d_list's progress: 5 page　　 71
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=5
d_list's progress: 7 page　　 86
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=7
d_list's progress: 9 page　　 108
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb

In [ ]:
df = pd.DataFrame(d_list)
df.head(10)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ相模線/南橋本,―徒歩5分,3.9,3000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t21,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築30年
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩7分,6,3000,6万円,6,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t22.35,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築13年
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/相模原,―徒歩10分,7.5,8000,7.5万円,7.5,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t31.22,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築3年
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩13分,5.1,3000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t18.31,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築5年
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区大島,京王相模原線/橋本,バス21分停歩4分,7,3500,7万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t48.55,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\n新築\n
5,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区橋本４,ＪＲ横浜線/橋本,―徒歩10分,3.7,2000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t17.39,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tワンルーム,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築32年
6,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ相模線/上溝,―徒歩12分,4,,-,4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t19.7,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築29年
7,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩29分,2.3,2000,-,2.3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t13.77,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tワンルーム,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築16年
8,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区...,ＪＲ相模線/南橋本,―徒歩13分,5.5,4000,2.75万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t26.08,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築6年
9,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区田名,ＪＲ横浜線/橋本,バス27分停歩8分,7.5,3500,2万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t70.1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築15年


In [ ]:
d_t_list = REDS(start+1,end+1,place)

d_list's progress: 2 page　　 19
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=2
d_list's progress: 4 page　　 45
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=4
d_list's progress: 6 page　　 63
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=6
d_list's progress: 8 page　　 82
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&kb=0&kt=9999999&km=1&xb=0&xt=9999999&et=9999999&cn=9999999&newflg=0&pn=8
d_list's progress: 10 page　　 110
https://suumo.jp/jj/common/ichiran/JJ901FC004/?ar=030&ta=14&sc=14151&sc=14152&sc=14153&kwd=&cb=0.0&ct=9999999&k

In [ ]:
df_test = pd.DataFrame(d_t_list)
df_test.head(10)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区田名,ＪＲ横浜線/橋本,バス16分停歩6分,6.4,3000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.12,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築22年
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区町屋４,ＪＲ横浜線/橋本,バス12分停歩3分,7.4,3000,7.4万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t56.81,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築4年
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩11分,6.4,3500,6.4万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t29.25,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築13年
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/相模原,―徒歩20分,7.3,5000,1万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t45.84,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築10年
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区相原３,ＪＲ横浜線/相原,―徒歩17分,8.2,6000,8.2万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.05,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築5年
5,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区橋本５,ＪＲ横浜線/橋本,―徒歩10分,10.3,5000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t67.25,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築23年
6,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上溝,ＪＲ相模線/上溝,―徒歩15分,6.4,4000,-,6.4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t59.69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築22年
7,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区相原４,ＪＲ横浜線/橋本,バス8分停歩5分,8.4,2500,8.4万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.78,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築8年
8,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩5分,6.2,1000,3.1万円,6.2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t20.28,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築11年
9,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/相模原,―徒歩15分,7.1,5000,-,14.2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.88,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3DK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築31年


In [ ]:
pprint(d_list)

[{'交通': '―徒歩5分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区清新５',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t21',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '3000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築30年',
  '賃料': '3.9',
  '路線': 'ＪＲ相模線/南橋本',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K'},
 {'交通': '―徒歩7分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区共和１',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t22.35',
  '敷金': '6',
  '礼金': '6万円',
  '管理共益費': '3000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築13年',
  '賃料': '6',
  '路線': 'ＪＲ横浜線/淵野辺',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K'},
 {'交通': '―徒歩10分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区相模原５',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t31.22',
  '敷金': '7.5',
  '礼金': '7.5万円',
  '管理共益費': '8000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築3年',
  '賃料': '7.5',
  '路線': 'ＪＲ横浜線/相模原',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK'},

  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区横山２',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t16',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '4000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築35年',
  '賃料': '2.5',
  '路線': 'ＪＲ相模線/上溝',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K'},
 {'交通': '―徒歩8分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区氷川町',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t35.5',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築33年',
  '賃料': '5.5',
  '路線': 'ＪＲ横浜線/相模原',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2DK'},
 {'交通': '―徒歩19分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区淵野辺本町４',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t55.5',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '3000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築26年',
  '賃料': '6.9',
  '路線': 'ＪＲ横浜線/淵野辺',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3DK'},
 {'交通': 'バス8分停歩5分',
  '住所': 

  '管理共益費': '2000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築19年',
  '賃料': '2.3',
  '路線': 'ＪＲ横浜線/淵野辺',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tワンルーム'},
 {'交通': '―徒歩20分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区相原４',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t49.47',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '4500',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築8年',
  '賃料': '7.9',
  '路線': 'ＪＲ横浜線/相原',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK'},
 {'交通': '―徒歩8分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区橋本２',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t24.52',
  '敷金': '',
  '礼金': '6.7万円',
  '管理共益費': '4500',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築7年',
  '賃料': '6.7',
  '路線': 'ＪＲ横浜線/橋本',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K'},
 {'交通': '―徒歩8分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区矢部４',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t16.58',
  '敷金': '',
  '礼金': '-',
  '

  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK'},
 {'交通': '―徒歩3分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区橋本２',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t23.79',
  '敷金': '6.8',
  '礼金': '6.8万円',
  '管理共益費': '',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築10年',
  '賃料': '6.8',
  '路線': 'ＪＲ横浜線/橋本',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K'},
 {'交通': '―徒歩10分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区矢部４',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t16.2',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '2000',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築30年',
  '賃料': '2.8',
  '路線': 'ＪＲ横浜線/矢部',
  '間取り': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K'},
 {'交通': '―徒歩21分',
  '住所': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上矢部４',
  '専有面積': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t19.87',
  '敷金': '',
  '礼金': '-',
  '管理共益費': '4500',
  '築年数': '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築15年',
  '賃料': '4.4',
  '路線': 'ＪＲ横浜線/淵野辺',
  '

In [ ]:
#dfの異なるデータの数を確認する
#print(len(df.住所.unique()))
#print(len(df.専有面積.unique()))
#print(len(df.交通.unique()))
print(df.duplicated().sum())
print(len(df))

0
208


In [ ]:
#重複した行を削除してみる
df = df.drop_duplicates()
df.head(300)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ相模線/南橋本,―徒歩5分,3.9,3000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t21,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築30年
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩7分,6,3000,6万円,6,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t22.35,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築13年
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/相模原,―徒歩10分,7.5,8000,7.5万円,7.5,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t31.22,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築3年
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩13分,5.1,3000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t18.31,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築5年
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区大島,京王相模原線/橋本,バス21分停歩4分,7,3500,7万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t48.55,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\n新築\n
5,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区橋本４,ＪＲ横浜線/橋本,―徒歩10分,3.7,2000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t17.39,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tワンルーム,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築32年
6,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ相模線/上溝,―徒歩12分,4,,-,4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t19.7,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築29年
7,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩29分,2.3,2000,-,2.3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t13.77,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tワンルーム,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築16年
8,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区...,ＪＲ相模線/南橋本,―徒歩13分,5.5,4000,2.75万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t26.08,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築6年
9,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区田名,ＪＲ横浜線/橋本,バス27分停歩8分,7.5,3500,2万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t70.1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築15年


In [ ]:
#df,df_test内の余計な文字(\n等)を消去する
df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
    df[element] = df[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':'','バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':'','築':'','年':'','新':'','万':'','(':'','円':''}))


df_test['礼金'] = df_test['礼金'].str.replace(r'\(.+\)','')
for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
    df_test[element] = df_test[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':'','バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':'','築':'','年':'','新':'','万':'','(':'','円':''}))
df.head(100)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,神奈川県相模原市中央区清５,ＪＲ相模線/南橋本,5,3.9,3000,,,21,1K,30
1,神奈川県相模原市中央区共和１,ＪＲ横浜線/淵野辺,7,6,3000,6,6,22.35,1K,13
2,神奈川県相模原市中央区相模原５,ＪＲ横浜線/相模原,10,7.5,8000,7.5,7.5,31.22,1LDK,3
3,神奈川県相模原市中央区相生１,ＪＲ横浜線/淵野辺,13,5.1,3000,,,18.31,1K,5
4,神奈川県相模原市緑区大島,京王相模原線/橋本,214,7,3500,7,,48.55,1LDK,
5,神奈川県相模原市緑区橋本４,ＪＲ横浜線/橋本,10,3.7,2000,,,17.39,ワンルーム,32
6,神奈川県相模原市中央区星が丘３,ＪＲ相模線/上溝,12,4,,,4,19.7,1K,29
7,神奈川県相模原市中央区青葉１,ＪＲ横浜線/淵野辺,29,2.3,2000,,2.3,13.77,ワンルーム,16
8,神奈川県相模原市緑区橋本台１,ＪＲ相模線/南橋本,13,5.5,4000,2.75,,26.08,1K,6
9,神奈川県相模原市中央区田名,ＪＲ横浜線/橋本,278,7.5,3500,2,,70.1,3LDK,15


In [ ]:
'''
#df内の余計な文字(\n等)を消去する
df['住所'] = df['住所'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))
df['交通'] = df['交通'].str.translate(str.maketrans({'バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':''}))
df['専有面積'] = df['専有面積'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))
df['間取り'] = df['間取り'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))
df['築年数'] = df['築年数'].str.translate(str.maketrans({'\r':'','\n':'','\t':'','築':'','年':'','新':''}))
df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
df['礼金'] = df['礼金'].str.translate(str.maketrans({'-':'','万':'','(':'','円':''}))
df['敷金'] = df['敷金'].str.translate(str.maketrans({'-':'','万':'','(':''}))
df.head(10)
'''

"\n#df内の余計な文字(\n等)を消去する\ndf['住所'] = df['住所'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))\ndf['交通'] = df['交通'].str.translate(str.maketrans({'バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':''}))\ndf['専有面積'] = df['専有面積'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))\ndf['間取り'] = df['間取り'].str.translate(str.maketrans({'\r':'','\n':'','\t':''}))\ndf['築年数'] = df['築年数'].str.translate(str.maketrans({'\r':'','\n':'','\t':'','築':'','年':'','新':''}))\ndf['礼金'] = df['礼金'].str.replace(r'\\(.+\\)','')\ndf['礼金'] = df['礼金'].str.translate(str.maketrans({'-':'','万':'','(':'','円':''}))\ndf['敷金'] = df['敷金'].str.translate(str.maketrans({'-':'','万':'','(':''}))\ndf.head(10)\n"

In [ ]:
#ラベルエンコーディングを行い、文字列を数値化する
LE1,LE2,LE3 = LabelEncoder(),LabelEncoder(),LabelEncoder()
#for i in ['住所','路線','間取り']:
    #df[i] = LE.fit_transform(df[i])
LE_df = pd.concat([df,df_test])
LE1.fit(LE_df['住所'])
LE2.fit(LE_df['路線'])
LE3.fit(LE_df['間取り'])

df['住所'] = LE1.transform(df['住所'])
df['路線'] = LE2.transform(df['路線'])
df['間取り'] = LE3.transform(df['間取り'])
df.head(10)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,42,11,5,3.9,3000,,,21,1,30
1,7,6,7,6,3000,6,6,22.35,1,13
2,50,8,10,7.5,8000,7.5,7.5,31.22,2,3
3,54,6,13,5.1,3000,,,18.31,1,5
4,77,1,214,7,3500,7,,48.55,2,
5,86,5,10,3.7,2000,,,17.39,12,32
6,26,10,12,4,,,4,19.7,1,29
7,62,6,29,2.3,2000,,2.3,13.77,12,16
8,82,11,13,5.5,4000,2.75,,26.08,1,6
9,44,5,278,7.5,3500,2,,70.1,9,15


In [ ]:
adressC,stationC,layoutC = LE1.classes_,LE2.classes_,LE3.classes_
print(adressC,'\n',stationC,'\n',layoutC)

['神奈川県相模原市中央区上溝' '神奈川県相模原市中央区上溝３' '神奈川県相模原市中央区上矢部１' '神奈川県相模原市中央区上矢部４'
 '神奈川県相模原市中央区下九沢' '神奈川県相模原市中央区中央５' '神奈川県相模原市中央区中央６' '神奈川県相模原市中央区共和１'
 '神奈川県相模原市中央区共和２' '神奈川県相模原市中央区共和４' '神奈川県相模原市中央区千代田１' '神奈川県相模原市中央区千代田２'
 '神奈川県相模原市中央区千代田５' '神奈川県相模原市中央区千代田７' '神奈川県相模原市中央区南橋本１' '神奈川県相模原市中央区南橋本２'
 '神奈川県相模原市中央区向陽町' '神奈川県相模原市中央区宮下本町１' '神奈川県相模原市中央区宮下本町２' '神奈川県相模原市中央区宮下本町３'
 '神奈川県相模原市中央区宮下２' '神奈川県相模原市中央区富士見１' '神奈川県相模原市中央区富士見２' '神奈川県相模原市中央区富士見３'
 '神奈川県相模原市中央区小山２' '神奈川県相模原市中央区小山３' '神奈川県相模原市中央区星が丘３' '神奈川県相模原市中央区東淵野辺３'
 '神奈川県相模原市中央区東淵野辺５' '神奈川県相模原市中央区横山２' '神奈川県相模原市中央区氷川町' '神奈川県相模原市中央区淵野辺本町１'
 '神奈川県相模原市中央区淵野辺本町２' '神奈川県相模原市中央区淵野辺本町３' '神奈川県相模原市中央区淵野辺本町４'
 '神奈川県相模原市中央区淵野辺１' '神奈川県相模原市中央区淵野辺２' '神奈川県相模原市中央区淵野辺３' '神奈川県相模原市中央区淵野辺４'
 '神奈川県相模原市中央区淵野辺５' '神奈川県相模原市中央区清１' '神奈川県相模原市中央区清３' '神奈川県相模原市中央区清５'
 '神奈川県相模原市中央区清６' '神奈川県相模原市中央区田名' '神奈川県相模原市中央区田名塩田２' '神奈川県相模原市中央区相模原１'
 '神奈川県相模原市中央区相模原２' '神奈川県相模原市中央区相模原３' '神奈川県相模原市中央区相模原４' '神奈川県相模原市中央区相模原５'
 '神奈川県相模原市中央区相模原６' '神奈川県相模原市中央区相模原７' '神奈川県相模原市中央区相模原８' '神奈川県相模原市中央区相生１'
 '神奈川県相模

In [ ]:
#欠損値をどうにかする
df.replace('',0,inplace=True)
df.head(10)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,42,11,5,3.9,3000,0,0,21,1,30
1,7,6,7,6,3000,6,6,22.35,1,13
2,50,8,10,7.5,8000,7.5,7.5,31.22,2,3
3,54,6,13,5.1,3000,0,0,18.31,1,5
4,77,1,214,7,3500,7,0,48.55,2,0
5,86,5,10,3.7,2000,0,0,17.39,12,32
6,26,10,12,4,0,0,4,19.7,1,29
7,62,6,29,2.3,2000,0,2.3,13.77,12,16
8,82,11,13,5.5,4000,2.75,0,26.08,1,6
9,44,5,278,7.5,3500,2,0,70.1,9,15


In [ ]:
df.head(len(df))

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,42,11,5,3.9,3000,0,0,21,1,30
1,7,6,7,6,3000,6,6,22.35,1,13
2,50,8,10,7.5,8000,7.5,7.5,31.22,2,3
3,54,6,13,5.1,3000,0,0,18.31,1,5
4,77,1,214,7,3500,7,0,48.55,2,0
5,86,5,10,3.7,2000,0,0,17.39,12,32
6,26,10,12,4,0,0,4,19.7,1,29
7,62,6,29,2.3,2000,0,2.3,13.77,12,16
8,82,11,13,5.5,4000,2.75,0,26.08,1,6
9,44,5,278,7.5,3500,2,0,70.1,9,15


In [ ]:
#データ型をfloat64で統一する。
df = df.astype('float64')
#df['礼金'] = df['礼金'].astype('float64')
print(df.dtypes)
df.head(10)

住所       float64
路線       float64
交通       float64
賃料       float64
管理共益費    float64
礼金       float64
敷金       float64
専有面積     float64
間取り      float64
築年数      float64
dtype: object


,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,42.0,11.0,5.0,3.9,3000.0,0.00,0.0,21.00,1.0,30.0
1,7.0,6.0,7.0,6.0,3000.0,6.00,6.0,22.35,1.0,13.0
2,50.0,8.0,10.0,7.5,8000.0,7.50,7.5,31.22,2.0,3.0
3,54.0,6.0,13.0,5.1,3000.0,0.00,0.0,18.31,1.0,5.0
4,77.0,1.0,214.0,7.0,3500.0,7.00,0.0,48.55,2.0,0.0
5,86.0,5.0,10.0,3.7,2000.0,0.00,0.0,17.39,12.0,32.0
6,26.0,10.0,12.0,4.0,0.0,0.00,4.0,19.70,1.0,29.0
7,62.0,6.0,29.0,2.3,2000.0,0.00,2.3,13.77,12.0,16.0
8,82.0,11.0,13.0,5.5,4000.0,2.75,0.0,26.08,1.0,6.0
9,44.0,5.0,278.0,7.5,3500.0,2.00,0.0,70.10,9.0,15.0


In [ ]:
#不動産賃料予測AIの学習
#Real_Estate_Rent_Learning
def RERL(df):
    #データを説明変数と目的変数に分ける
    target = df['賃料']
    train = df.drop(['賃料'],axis=1)
    
    #ホールドアウト法で学習データとテストデータに分ける
    kf = KFold(n_splits=4,shuffle=True,random_state=71)
    tr_idx,va_idx = list(kf.split(train))[0]
    tr_x,va_x = train.iloc[tr_idx],train.iloc[va_idx]
    tr_y,va_y = target.iloc[tr_idx],target.iloc[va_idx]
    
    #データ型をXGBOOST用に適合させる
    dtrain = xgb.DMatrix(tr_x,label=tr_y)
    dvalid = xgb.DMatrix(va_x,label=va_y)
    dtest = xgb.DMatrix(train)
    
    #パラメータ準備
    params = {'objective':'reg:squarederror','silent':1,'random_state':71}
    num_round = 100

    #XGBOOSTモデルに機械学習
    watchlist = [(dtrain,'train'),(dvalid,'eval')]
    model = xgb.train(params,dtrain,num_round,early_stopping_rounds=10,evals=watchlist)
    va_pred = model.predict(dvalid)
    
    #テストデータとその予測結果を表示
    print(list(va_y))
    print(va_pred)
    
    #モデルの性能を表示
    print('MAE:',mean_absolute_error(va_y,va_pred))
    print('MSE:',mean_squared_error(va_y,va_pred))
    print('RMSE:',np.sqrt(mean_squared_error(va_y,va_pred)))
    
    return model


In [ ]:
model = RERL(df)

[20:25:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:4.14151	eval-rmse:3.94353
[1]	train-rmse:3.00904	eval-rmse:2.82583
[2]	train-rmse:2.20630	eval-rmse:2.07853
[3]	train-rmse:1.63248	eval-rmse:1.60405
[4]	train-rmse:1.22115	eval-rmse:1.29281
[5]	train-rmse:0.91968	eval-rmse:1.14594
[6]	train-rmse:0.69519	eval-rmse:1.06711
[7]	train-rmse:0.53076	eval-rmse:0.99093
[8]	train-rmse:0.41181	eval-rmse:0.95146
[9]	train-rmse:0.32271	eval-rmse:0.93316
[10]	train-rmse:0.25948	eval-rmse:0.91658
[11]	train-rmse:0.21521	eval-rmse:0.90991
[12]	train-rmse:0.18217	eval-rmse:0.90476
[13]	train-rmse:0.15755	eval-rmse:0.90174
[14]	train-rmse:0.14043	eval-rmse:0.90546
[15]	train-rmse:0.1258

In [ ]:
df_test.head(10)

,住所,路線,交通,賃料,管理共益費,礼金,敷金,専有面積,間取り,築年数
0,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区田名,ＪＲ横浜線/橋本,バス16分停歩6分,6.4,3000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.12,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築22年
1,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区町屋４,ＪＲ横浜線/橋本,バス12分停歩3分,7.4,3000,7.4万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t56.81,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築4年
2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩11分,6.4,3500,6.4万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t29.25,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築13年
3,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/相模原,―徒歩20分,7.3,5000,1万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t45.84,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築10年
4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区相原３,ＪＲ横浜線/相原,―徒歩17分,8.2,6000,8.2万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.05,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築5年
5,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区橋本５,ＪＲ横浜線/橋本,―徒歩10分,10.3,5000,-,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t67.25,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築23年
6,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上溝,ＪＲ相模線/上溝,―徒歩15分,6.4,4000,-,6.4,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t59.69,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築22年
7,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市緑区相原４,ＪＲ横浜線/橋本,バス8分停歩5分,8.4,2500,8.4万円,,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.78,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t2LDK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築8年
8,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/淵野辺,―徒歩5分,6.2,1000,3.1万円,6.2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t20.28,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t1K,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築11年
9,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央...,ＪＲ横浜線/相模原,―徒歩15分,7.1,5000,-,14.2,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t58.88,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3DK,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t築31年


In [ ]:
#学習データを作成する関数
#Real_Estate_Data_Preprocessing
def REDP(d_list):
    #データフレームに変換
    df = pd.DataFrame(d_list)
    
    #df内の余計な文字(\n等)を消去する
    df['礼金'] = df['礼金'].str.replace(r'\(.+\)','')
    for element in ['住所','交通','専有面積','間取り','築年数','礼金','敷金']:
        df[element] = df[element].str.translate(str.maketrans({'-':'','\r':'','\n':'','\t':'','バ':'','ス':'','分':'','徒':'','歩':'','停':'','―':'','築':'','年':'','新':'','万':'','(':'','円':''}))
    
    #ラベルエンコーディングを行い、文字列を数値化する
    #ラベルエンコーディングのモデルを初期化していたから、本来の力を発揮できていなかった
    #LE = LabelEncoder()
    df['住所'] = LE1.transform(df['住所'])
    df['路線'] = LE2.transform(df['路線'])
    df['間取り'] = LE3.transform(df['間取り'])
    
    #欠損値をどうにかする
    df.replace('',0,inplace=True)
    
    #データ型をfloat64で統一する。
    df = df.astype('float64')
    
    #入力データと正解データに分ける
    #Fact = df['賃料']
    #Input = df.drop(['賃料'],axis=1)
    
    return df


In [ ]:
df_t = REDP(d_t_list)
df_t.head(10)

TypeError: REDP() missing 1 required positional argument: 'd_t_list'

In [ ]:
print('df_test:{}'.format(len(df_test['住所'].unique())))
print('df:{}'.format(len(df['住所'].unique())))

df_test:73
df:78


In [ ]:
#SBRLで作ったXGBOOSTモデルを用いて、df_testを入力として賃料を予測する関数を作る
#Real_Estate_Rent_Divided_Prediction
def RERDP(df):
    #説明変数と目的変数の分離
    df_fact = df['賃料']
    df_input = df.drop(['賃料'],axis=1)
    
    #予測値の生成
    df_input = xgb.DMatrix(df_input)
    pred = model.predict(df_input)
    
    #実値と予測値の表示
    #df_fact = df_fact.rename({name:'実値'})
    df_pred = pd.DataFrame(pred,columns=['予測値'])
    df_final = pd.concat([df_fact,df_pred],axis=1)
    print(df_final)
    
    #モデルの性能を表示
    print('MAE:',mean_absolute_error(df_fact,df_pred))
    print('MSE:',mean_squared_error(df_fact,df_pred))
    print('RMSE:',np.sqrt(mean_squared_error(df_fact,df_pred)))


In [ ]:
RERDP(df_t)

        賃料        予測値
0     6.40   6.964079
1     7.40   7.480511
2     6.40   6.358985
3     7.30   7.254192
4     8.20   7.685145
5    10.30   8.418609
6     6.40   7.153454
7     8.40   7.630031
8     6.20   5.670850
9     7.10   7.377625
10    8.40   7.067617
11    6.10   5.777291
12    4.90   5.677645
13    4.60   4.407328
14    6.00   6.693394
15    7.70   7.679375
16    7.70   6.716848
17    7.60   7.679375
18    7.50   6.389239
19    6.70   6.733297
20    2.90   3.809378
21    8.40   7.630031
22    2.80   4.009113
23    5.10   6.446550
24   11.10   8.124542
25    2.80   4.682752
26    5.00   4.023717
27    5.70   6.145385
28    6.20   7.293133
29    5.20   5.568092
30    7.40   6.544883
31    4.80   4.900165
32    6.50   6.626788
33    5.00   5.310926
34    4.70   4.025902
35    7.55   7.446887
36    4.60   4.474434
37    3.70   3.916200
38   10.40   8.080600
39    7.40   8.348355
40    6.20   5.683607
41    6.30   6.849229
42    8.45   7.535977
43    7.40   7.857271
44    5.90

In [ ]:
adressC,stationC,layoutC = LE1.classes_,LE2.classes_,LE3.classes_
print(adressC,'\n','\n',stationC,'\n','\n',layoutC)

['\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上溝'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上溝３'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上矢部１'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区上矢部４'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区下九沢'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区中央５'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区中央６'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区共和１'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区共和２'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区共和４'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区千代田１'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区千代田２'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区千代田５'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区千代田７'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区南橋本１'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区南橋本２'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区向陽町'
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t神奈川県相模原市中央区宮下本町１'
 '\r\n\t\t\t\t\t\t\

In [ ]:
#お待ちかねのオリジナル予測！
#学習した範囲内の任意のデータを入力して予測できる！
#Real_Estate_Own_Data_Prediction
def REODP(address,station,access,mc_fees,k_fees,s_fees,area,layout,age):
    d_try = {
        '住所':address,
        '路線':station,
        '交通':access,
        '管理共益費':mc_fees,
        '礼金':k_fees,
        '敷金':s_fees,
        '専有面積':area,
        '間取り':layout,
        '築年数':age
    }
    df_try = pd.DataFrame(d_try,index=['own'])
    display(df_try)

    df_try.住所 = LE1.transform(df_try.住所)
    df_try.路線 = LE2.transform(df_try.路線)
    df_try.間取り = LE3.transform(df_try.間取り)
    display(df_try)
    
    df_try = df_try.astype('float64')
    df_try = xgb.DMatrix(df_try)
    
    return print(model.predict(df_try))


In [ ]:
REODP(
    address=''
    ,station=''
    ,access=0
    ,mc_fees=0
    ,k_fees=0
    ,s_fees=0
    ,area=0
    ,layout=''
    ,age=0
)